# MAP543 - Database Management
Students: Khouloud EL ALAMI, Aya ERRAJRAJI, Ali EL ABBASSY
***
CCF: Fast and Scalable Connected Component Computation in MapReduce
https://www.cse.unr.edu/~hkardes/pdfs/ccf.pdf

In [0]:
import time
import pandas as pd
import pyspark

In [0]:
# Toy graph from the paper
# 8 nodes and 6 edges
toy_graph = sc.parallelize([
  ('A','B'),
  ('B','C'),
  ('B','D'),
  ('D','E'),
  ('F','G'),
  ('G','H'),
])

# Google web graph from http://snap.stanford.edu/data/web-Google.html
# 875K nodes and 5.1M edges
rdd_google = sc.textFile("/FileStore/tables/web_Google.txt").cache()
rdd_google = rdd_google.filter(lambda x : '#' not in x).map(lambda x : x.split('\t')).map(lambda x: (int(x[0]), int(x[1])))

In [0]:
def CCF_iterate_map(rdd):
    reverse_rdd = rdd.map(lambda x: (x[1], x[0]))
    emit = rdd.union(reverse_rdd)
    emit = emit.groupByKey().mapValues(list)
    return emit.partitionBy(10)
  
def CCF_iterate_reduce(mapping):
    # Since we only emit if the minimum value is smaller than the key, filtering whenever it's not the case
    mapping = mapping.map(lambda x : (x[0], x[1], min(x[0], min(x[1]))))
    mapping = mapping.filter(lambda x: x[0] != x[2])
    # First emit
    emit1 = mapping.map(lambda x: (x[0], x[2]))
    # Only keeping the values that are different from the minimum
    mapping = mapping.map(lambda x: (x[0], [value for value in x[1] if value != x[2]], x[2]))
    # Updating the global NewPair counter
    CounterNewPair = mapping.map(lambda x: len(x[1])).sum()
    # Second emit
    emit2 = mapping.flatMap(lambda x: [(value, x[2]) for value in x[1]])
    emit = emit1.union(emit2)
    return emit, CounterNewPair
  
def CCF_iterate_ss_reduce(mapping):
    # Passing the values to the reducer in a sorted way
    mapping = mapping.map(lambda x: (x[0], sorted(x[1])))
    # Since we only emit if the minimum value is smaller than the key, we filter whenever it's not the case
    mapping = mapping.filter(lambda x: x[0] > x[1][0])
    # First emit
    emit1 = mapping.map(lambda x: (x[0], x[1][0]))
    # Only keeping the values that are different from the minimum since its pair has already been emitted
    mapping = mapping.map(lambda x: (x[0], x[1][1:], x[1][0]))
    # Updating the global NewPair counter
    CounterNewPair = mapping.map(lambda x: len(x[1])).sum()
    # Second emit
    emit2 = mapping.flatMap(lambda x: [(value, x[2]) for value in x[1]])
    emit = emit1.union(emit2)
    return emit, CounterNewPair
  
def CCF(rdd, secondary_sorting):
    iteration = 0
    CounterNewPair = 1 # >0 to allow entering the while loop
    while CounterNewPair > 0:
        mapping = CCF_iterate_map(rdd.distinct()) # The CCF-Dedup job is done by distinct()
        if not secondary_sorting:
            rdd, CounterNewPair = CCF_iterate_reduce(mapping)
        else:
            rdd, CounterNewPair = CCF_iterate_ss_reduce(mapping)
        iteration += 1
    return rdd, iteration

#### Experiments

##### I) Toy Graph

In [0]:
start = time.time()
output_toy, nb_iterations_1 = CCF(toy_graph, secondary_sorting=False)
timing_1 = time.time() - start
print(f'Number of iterations: {nb_iterations_1}')
print(f'Run-time: {timing_1:.2f} (sec)')

nb_cc = output_toy.map(lambda x : x[1]).distinct().count()
nodes_largest = output_toy.groupBy(lambda x : x[1]).map(lambda x : len(x[1])).max() + 1
print(f"There are {nb_cc} connected components in this graph")
print(f"There are {nodes_largest} nodes in the largest connected component")

Number of iterations: 4
Run-time: 8.65 (sec)
There are 2 connected components in this graph
There are 5 nodes in the largest connected component

In [0]:
start = time.time()
output_toy, nb_iterations_2 = CCF(toy_graph, secondary_sorting=True)
timing_2 = time.time() - start
print(f'Number of iterations: {nb_iterations_2}')
print(f'Run-time: {timing_2:.2f} (sec)')

nb_cc = output_toy.map(lambda x : x[1]).distinct().count()
nodes_largest = output_toy.groupBy(lambda x : x[1]).map(lambda x : len(x[1])).max() + 1
print(f"There are {nb_cc} connected components in this graph")
print(f"There are {nodes_largest} nodes in the largest connected component")

Number of iterations: 4
Run-time: 7.27 (sec)
There are 2 connected components in this graph
There are 5 nodes in the largest connected component

##### II) Google Graph

In [0]:
start = time.time()
output_google, nb_iterations_3 = CCF(rdd_google, secondary_sorting=False)
timing_3 = time.time() - start
print(f'Number of iterations: {nb_iterations_3}')
print(f'Run-time: {timing_3:.2f} (sec)')

nb_cc = output_google.map(lambda x : x[1]).distinct().count()
nodes_largest = output_google.groupBy(lambda x : x[1]).map(lambda x : len(x[1])).max() + 1
print(f"There are {nb_cc} connected components in this graph")
print(f"There are {nodes_largest} nodes in the largest connected component")

Number of iterations: 8
Run-time: 578.57 (sec)
There are 2746 connected components in this graph
There are 855802 nodes in the largest connected component

In [0]:
start = time.time()
output_google, nb_iterations_4 = CCF(rdd_google, secondary_sorting=True)
timing_4 = time.time() - start
print(f'Number of iterations: {nb_iterations_4}')
print(f'Run-time: {timing_4:.2f} (sec)')

nb_cc = output_google.map(lambda x : x[1]).distinct().count()
nodes_largest = output_google.groupBy(lambda x : x[1]).map(lambda x : len(x[1])).max() + 1
print(f"There are {nb_cc} connected components in this graph")
print(f"There are {nodes_largest} nodes in the largest connected component")

Number of iterations: 8
Run-time: 567.73 (sec)
There are 2746 connected components in this graph
There are 855802 nodes in the largest connected component

#### Results

In [0]:
perf_toy = {'Number of iterations':  [nb_iterations_1, nb_iterations_2],
        'Run-time (sec)': [timing_1, timing_2]}
perf_toy = pd.DataFrame (perf_toy, columns=['Number of iterations','Run-time (sec)'], index=['CCF w/o sec. sorting', 'CCF w/ sec. sorting'])
perf_toy

Out[26]:

,Number of iterations,Run-time (sec)
CCF w/o sec. sorting,4,8.654075
CCF w/ sec. sorting,4,7.268254


In [0]:
perf_google = {'Number of iterations':  [nb_iterations_3, nb_iterations_4],
        'Run-time (sec)': [timing_3, timing_4]}
perf_google = pd.DataFrame(perf_google, columns=['Number of iterations', 'Run-time (sec)'], index=['CCF w/o sec. sorting', 'CCF w/ sec. sorting'])
perf_google

Out[27]:

,Number of iterations,Run-time (sec)
CCF w/o sec. sorting,8,578.569356
CCF w/ sec. sorting,8,567.727934
